# Generators

## Lesson Objectives
By the end of this lesson, you will be able to:
- Understand the generator use case
- Construct a generator
- Yield data from, and send data to, generator functions
- Use generator comprehensions

## Table of Contents
 - [Why Generators?](#why)
 - [Creating a Generator](#generator)
 - [Generator Comprehensions](#gc)
 - [Takeaways](#takeaways)
 - [What's Next](#next)
 - [Applications](#applications)

<a id='why' ></a>
## Why Generators?

#### The Python Function as a Subroutine
As we know, Python functions `return` a value. But think for a moment about why we say "`return`." You might recall that, when we call a function, code execution starts at the function's first line and continues until we encounter a `return` statement, `Exception`, or the end of the function, in which case `None` is actually returned. Then, once our function is done, all of its local variables (its state) are lost and we **return** the program to the point at which the function was called. Any new calls to the function would repeat this process. In computer programming, this process is known as a [subroutine](https://en.wikipedia.org/wiki/Subroutine).

That's all fine and dandy, but once you start working with larger streams of data, there will be times when you want a function that can *yield a series of values* instead of returning a single value and then wiping its state. For a function to do this, it would need to be able to remember its state between each yield.

Why did I italicize *yield a series of values* in the paragraph above? I did so because our hypothetical function wouldn't `return` like a typical Python function. As a subroutine, a function `return` means that the function is returning control of code execution to the point in our program where the function was called. `yield` - the Python keyword for defining generators - makes that transfer of control temporary, leaving our function's state intact and expectant of regaining control again in the future.

**Functions that `yield` a series of values are called generators in Python.** Before generators, creating large sequences of data required code that both generated values and kept track of the function's state between calls. Since generators remember their state so to speak, yielding a series of values has become much easier.

#### Generators are a Subtype of Iterators
Generators can yield a series of values because they're functions that act like iterators. In fact, they're a subtype of iterators. But what's an iterator? Let's unpack that real quick:

> **Iteration** is the process of doing something to each item in a sequence of items. Using a loop is an example of iteration. This term is not specific to Python (it's basis is in mathematics).

> **Iterable** and **iterator** have specific meanings in Python:

>> An **iterator** is an object representing a stream of data; this object returns the data one element at a time with the `__next__` method until there are no more elements in the stream, in which case the object raises the `StopIteration` exception.

>> An **iterable** is an object that has an `__iter__` method, which returns an iterator, or which defines a  `__getitem__` method that can take sequential indexes starting from zero, raising an `IndexError` when the index exceeds the length of the sequence.

>> In short, **an iterable is an object that you can get an iterator from**.

**Iterators are great because they save memory.** When you instantiate an iterator, the object doesn't contain the whole sequence in memory and will only compute the values from the sequence when you ask for them. This is known as [lazy evaluation](https://en.wikipedia.org/wiki/Lazy_evaluation) and it's a boon when you're working with large (or infinite!) sequences of data. 

### Using an Iterator to Work with Prime Numbers
To see iterators in action, we'll write some code to get the sum of all of the prime numbers smaller than a given number. We'll do this two ways:  with and without an iterator. But first, let's define a helper function that tests whether a given number is prime or not using the [trial division primality test](https://en.wikipedia.org/wiki/Primality_test#Simple_methods).

In [42]:
def primality_test(number):
    """
    Return a boolean after peforming the trial division primality test on a number.
   
    Trial Division Primality Test:  Given an input number n, check whether any prime integer m 
                                    from 2 to the square root of n evenly divides n.
                                    If n is divisible by any m then n is composite, otherwise it is prime.
    """
    
    for divisor in range(2, int(number ** 0.5) + 1):
        if number % divisor == 0:
            return False
    return True

#### Without an Iterator
Now that we have a function that tests whether or not a number is prime, let's get all of the prime numbers below `1,000,000` using a conventional `for` loop approach.

In [44]:
%time
primes = []
for n in range(2,1000000):
    if primality_test(n):
        primes.append(n)
    else:
        pass

answer = sum(primes)

print("The sum is:  {}".format(answer))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
The sum is:  37550402023


So that was pretty straightforward. Unfortunately, we now have a list of `78,498` elements in our memory. That might not be limiting right now, but what if we wanted to get the sum of all the primes below a much larger number?


So let's try this using an iterator.

#### With an Iterator
To turn our function `primality_test` into an iterator, we need to put it into a class and then define an `__iter__` and `__next__` method.

In [45]:
class Primes:
    
    def __init__(self, max_number):
        self.max_number = max_number
        self.number = 1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.number += 1
        if self.number >= self.max_number:
            raise StopIteration
        elif primality_test(self.number):
            return self.number
        else:
            return self.__next__()
        

In [46]:
%time
primes = Primes(1000000)
sum(primes)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.91 µs


37550402023

Although the iterator approach saves us on memory, it might not have saved us on production time since we had to write a bit more code. Generators were introduced in part to meet this challenge.

<a id='generator' ></a>
## Creating a Generator
If the body of a function contains the `yield` keyword, you have yourself a generator, even if it also contains a `return` statement! Let's define a simple one:

In [47]:
def generator_function():
    print("Here we go!") #you'll see why I included this print statement shortly.
    yield 1
    yield 2
    yield 3

Calling a generator function returns a generator object, which is a special type of iterator. Once you have that generator object, you can loop through it (which implicitly calls the `__next__` method) or use the built-in `next()` function.

In [49]:
generator = generator_function()

In [50]:
for n in generator:
    print(n)

Here we go!
1
2
3


In [51]:
generator = generator_function()

In [52]:
next(generator)

Here we go!


1

In [53]:
next(generator)

2

In [54]:
next(generator)

3

So what's going on up there?

When we created a generator object, the generator function's code was not run all at once (notice how the print statement didn't execute with assigment `generator = generator_function()`). 

Only when we made calls to `next` (or `__next__`) did "part of" the generator function's code execute. I say "part of" because, as you can see, code execution in a generator stops once a `yield` statement has been reached. So the first `next` call yielded the first value. The second call to `next` then **resumed execution with the state in which the generator had after the first `yield`**. This is the fundamental difference between generators and regular functions: regular functions always start code execution at the top of the function definition and then discard their state once they `return` (or hit an `Exception`). Generators don't discard their state; they suspend it until `next` is called on them again.

In short, generator functions return an object on which you can call `next`, such that for every call it returns the next value, until it raises a `StopIteration` exception, signaling that all of the values within the iterator have been generated:

In [55]:
next(generator)

StopIteration: 

### Using a Generator to Sum Primes
Now that we have a handle on generators, let's rewrite our code to sum all those prime numbers using a generator function.

In [56]:
def prime_sum_gen(max_number):
    n = 1
    while n < max_number:
        n += 1
        if primality_test(n):  # <-- there's the primality test function
            yield n

In [57]:
primes = prime_sum_gen(1000000)

In [58]:
%time
sum(primes)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


37550402023

There you have it! And, believe it or not, we can improve on that code with a **generator expression**, which we'll see next.

## Generator Expressions
Remember list comprehensions? Well, there are also [generator expressions](https://www.python.org/dev/peps/pep-0289/), which allow you to quickly and easily build a basic generator. To create them, you use parentheses instead of square brackets.

In [59]:
gen = (n for n in range(5))
for n in gen:
    print(n)

0
1
2
3
4


And now to get the sum of all those primes using a generator comprehension.

In [60]:
sum(n for n in range(2,1000000) if primality_test(n))

37550402023

<a id = 'takeaways'> </a>
## Takeaways
Here are the key things to remember about generators:

 - generators yield a series of values
 - the `yield` keyword coverts a function to a generator
 - A generator is just a special type of iterator, which means we can get its next value using `next()` or a loop

<a id = 'next'> </a>
## What's Next
Generators can also receive a value. This opens the door to writing [coroutines](https://en.wikipedia.org/wiki/Coroutine), which can be used for all sorts of cool things like asynchronous IO multitasking. We'll (maybe) tackle these topics in a later lesson.

<a id='applications'> </a>
## Applications
When your programs become larger, you'll ultimately want to make the output of one function the input to another. You might even want to do this several times. Chaining processes together like this is called piping. As we'll soon see, generators work will within this framework.

### Reading Lots of Large Files
Generators are great for reading a large number of large files since they yield one data point at a time insteading of building up a list of data points in memory.

To see this in action, we'll use both a loop (the conventional approach) and then a pipe of generators to extract a text pattern from a directory containing ten [Project Gutenberg](https://www.gutenberg.org/) books (which I've conventiently placed in an assets folder for you).

#### The `for` Loop Approach

In [65]:
import os

def pattern_finder(pattern="pattern"):
    matches = []
    for dir_path, dir_names, file_names in os.walk('assets/'):
        for file_name in file_names:
            if file_name.endswith('.txt'):
                for line in open(os.path.join(dir_path, file_name)):
                    if pattern in line:
                        matches.append(line)
    return matches

In [66]:
pattern_matches = pattern_finder()
for match in pattern_matches:
    print(match)

most abundant were raked into a gridiron-pattern by fingers, these

present mode. I never saw the mode. I have had a pattern in my hand.” He

of wine, Madame Defarge herself picked out the pattern on her sleeve

“A pretty pattern too!”

pattern. Enough! Now, my dear Lucie,” drawing his arm soothingly round

of his pate shaved into queer patterns, and three ornamental scars

faces, and the pattern on their backs was the same as the rest of the

One of those sprawling flamboyant patterns committing every artistic sin.

There is a recurrent spot where the pattern lolls like a broken neck and

This wallpaper has a kind of sub-pattern in a different shade, a

follow that pattern about by the hour. It is as good as gymnastics, I

time that I will follow that pointless pattern to some sort of a

Behind that outside pattern the dim shapes get clearer every day.

pattern. I don’t like it a bit. I wonder—I begin to think—I

The faint figure behind seemed to shake the pattern, just as if she want

Our function above looped through all of the files in our assets directory. If the file name ended with `.txt`, the function opened the file and then looped through each line looking for the word `pattern`. If it saw that word, it appended the line to `matches`. Then we used a loop to print the matches.

As you saw, this function worked well for our ten books. But that's just because we had a small number of small files. If we had more files -  and therefore more matches - our `matches` list might eventually exceed our memory limits. Another point to consider is that reading nested loops can quickly become difficult.

#### The Generator Pipeline Approach
To pipe our program, we'll divide our process into three different components, each of which will rely on a generator:
 1. generating the filenames
 2. generating the lines
 3. generating the matches
 
The output of the first pipe will be the input of the second and so forth.

In [67]:
def generate_filenames():
    for dir_path, dir_names, file_names in os.walk('assets/'):
        for file_name in file_names:
            if file_name.endswith('.txt'):
                yield open(os.path.join(dir_path, file_name))

def generate_lines(files):
    for fname in files:
        for line in fname:
            yield line

def generate_matches(lines, pattern=None):
    for line in lines:
        if pattern in line:
            yield line


book_files = generate_filenames()
book_lines = generate_lines(book_files)
pattern_matches = generate_matches(book_lines, pattern = 'pattern')

for match in pattern_matches:
    print(match)

most abundant were raked into a gridiron-pattern by fingers, these

present mode. I never saw the mode. I have had a pattern in my hand.” He

of wine, Madame Defarge herself picked out the pattern on her sleeve

“A pretty pattern too!”

pattern. Enough! Now, my dear Lucie,” drawing his arm soothingly round

of his pate shaved into queer patterns, and three ornamental scars

faces, and the pattern on their backs was the same as the rest of the

One of those sprawling flamboyant patterns committing every artistic sin.

There is a recurrent spot where the pattern lolls like a broken neck and

This wallpaper has a kind of sub-pattern in a different shade, a

follow that pattern about by the hour. It is as good as gymnastics, I

time that I will follow that pointless pattern to some sort of a

Behind that outside pattern the dim shapes get clearer every day.

pattern. I don’t like it a bit. I wonder—I begin to think—I

The faint figure behind seemed to shake the pattern, just as if she want

Above, we did not use any extra variables to print our matches. Instead, we created a pipeline that fed its components via the iteration process one item at a time. `generate_matches` took in a generator object of all the book_lines of all of the `.txt` files. Similarly, `generate_lines` took in a generator object of all the filenames in a directory.

The takeaway is that generator pipelines are a great way to break apart complex programs into smaller pieces.